In [24]:
import pandas as pd
pd.set_option('max_columns', 500)
from os import listdir
from os.path import isfile, join
from datetime import datetime, timedelta

In [38]:
path = './data/Medical/'

In [39]:
datafiles = [f for f in listdir(path) if isfile(join(path, f)) and not '.xlsx' in f and not 'medical' in f]
for i, file in enumerate(datafiles):
    if i == 0:
        df = pd.read_csv(path + datafiles[i])
    else:
        if i == 3:
            df_new = pd.read_csv(path + datafiles[3], names=df.columns, skiprows=0).drop(0).reset_index().drop(columns=['index'])
        else:
            df_new = pd.read_csv(path + datafiles[i])
        df = pd.concat([df, df_new])

/nfs/home/iglukhov/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/nfs/home/iglukhov/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/nfs/home/iglukhov/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,17,18,19,20,21,22,23,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,48,49,50,51,52,53) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/nfs/home/iglukhov/anaconda3/lib/python3.8/site-packages/IPython/core/interac

In [40]:
shape_before = df.shape[0]
df = df[(df['state_code'] != 'NOT AT HOME') & (df['state_code'] != 'MARRIED')]
df = df[(df['identification_code'] != 'S HOUSE') & (df['identification_code'] != 'MARRIED')]

df = df[(df['state_code'].notna()) & (df['identification_code'].notna())& (df['day_or_month_for_breast_feeding'].notna())]
df = df.drop(columns=['record_code_iodine_reason', 'usual_residance_reason', 'psu_id', 'identification_code'])

print(f'shape dropped by {-100* (df.shape[0] - shape_before) / shape_before}%')

df['v54'] = df['v54'].fillna(1)

df['day_or_month_for_breast_feeding_'] = df['day_or_month_for_breast_feeding_'].apply(lambda x: '-1' if x == -1 else x)
df = pd.concat([df.drop(columns=['day_or_month_for_breast_feeding_']), pd.get_dummies(df['day_or_month_for_breast_feeding_'], prefix='day_or_month_for_breast_feeding_')], axis=1)

df = pd.concat([df, pd.get_dummies(df['length_height_code'], prefix='length_height_code')], axis=1).drop(columns=['length_height_code'])
df = pd.concat([df, pd.get_dummies(df['age_code'], prefix='age_code')], axis=1).drop(columns=['age_code'])

df['date_survey'] = pd.to_datetime(df['date_survey'], infer_datetime_format=True)
date_max = df['date_survey'].max()
df['date_survey'] = df['date_survey'].apply(lambda x: (date_max - x))
df['date_survey'] = df['date_survey'].dt.days

numeric_cols = ['state_code', 'district_code', 'rural_urban', 'stratum',
       'ahs_house_unit', 'house_hold_no', 'test_salt_iodine',
       'record_code_iodine', 'sl_no', 'sex',
       'usual_residance', 'age', 'date_of_birth', 'month_of_birth', 
       'year_of_birth',
       'weight_measured', 'weight_in_kg', 'length_height_measured',
                'length_height_cm', 'haemoglobin_test',
       'haemoglobin', 'haemoglobin_level', 'bp_systolic',
       'bp_systolic_2_reading', 'bp_diastolic', 'bp_diastolic_2reading',
       'pulse_rate', 'pulse_rate_2_reading', 'diabetes_test',
       'fasting_blood_glucose', 'fasting_blood_glucose_mg_dl',
       'marital_status', 'gauna_perfor_not_perfor', 'duration_pregnanacy',
       'first_breast_feeding', 'is_cur_breast_feeding',
       'water_month', 'ani_milk_month', 'semisolid_month_or_day',
       'solid_month', 'vegetables_month_or_day', 'illness_type',
       'illness_duration', 'treatment_type', 'v54', 
               'day_or_month_for_breast_feeding']

for c in numeric_cols:
    print(c)
    df[c] = df[c].astype(float)
    
df = df[(df['illness_type'] != 11) & (df['illness_type'] != -1)]
df = df.rename(columns={'illness_type': 'target'})

features = [elem for elem in df.columns if elem != 'target']
df[features]=(df[features]-df[features].mean())/df[features].std()
df = df[df['target'].notna()]
df = df.reset_index().drop(columns=['index'])
df['target'] = df['target'].astype(int)

shape dropped by 0.0303778112275025%
state_code
district_code
rural_urban
stratum
ahs_house_unit
house_hold_no
test_salt_iodine
record_code_iodine
sl_no
sex
usual_residance
age
date_of_birth
month_of_birth
year_of_birth
weight_measured
weight_in_kg
length_height_measured
length_height_cm
haemoglobin_test
haemoglobin
haemoglobin_level
bp_systolic
bp_systolic_2_reading
bp_diastolic
bp_diastolic_2reading
pulse_rate
pulse_rate_2_reading
diabetes_test
fasting_blood_glucose
fasting_blood_glucose_mg_dl
marital_status
gauna_perfor_not_perfor
duration_pregnanacy
first_breast_feeding
is_cur_breast_feeding
water_month
ani_milk_month
semisolid_month_or_day
solid_month
vegetables_month_or_day
illness_type
illness_duration
treatment_type
v54
day_or_month_for_breast_feeding


In [41]:
df['target'].value_counts()

0    67631
2    10212
3     6302
1     5941
4     1180
5      350
Name: target, dtype: int64

In [42]:
df_class_0_under = df[df['target'] == 0].sample(5000, replace=False, random_state=0)
df = pd.concat([df_class_0_under, df[df['target'] != 0]], axis=0)

In [43]:
df_class_2_under = df[df['target'] == 2].sample(5000, replace=False, random_state=0)
df = pd.concat([df_class_2_under, df[df['target'] != 2]], axis=0)

In [44]:
df_class_3_under = df[df['target'] == 3].sample(5000, replace=False, random_state=0)
df = pd.concat([df_class_3_under, df[df['target'] != 3]], axis=0)

In [45]:
df_class_1_under = df[df['target'] == 1].sample(5000, replace=False, random_state=0)
df = pd.concat([df_class_1_under, df[df['target'] != 1]], axis=0)

In [46]:
df_class_4_over = df[df['target'] == 4].sample(5000, replace=True, random_state=0)
df = pd.concat([df_class_4_over, df[df['target'] != 4]], axis=0)

In [47]:
df_class_5_over = df[df['target'] == 5].sample(5000, replace=True, random_state=0)
df = pd.concat([df_class_5_over, df[df['target'] != 5]], axis=0)

In [48]:
df.shape

(30000, 60)

In [51]:
int(0.99 * len(df))

29700

In [52]:
len(df) - int(0.99 * len(df)) 

300

In [ ]:
73292 - 300 * (73292 // 300)

In [49]:
df['target'].value_counts()

0    5000
1    5000
2    5000
3    5000
4    5000
5    5000
Name: target, dtype: int64

In [ ]:
df

In [50]:
df.to_csv(path + 'medical_preprocessed.csv', index=False)